## Set Similarity Model

The following codeblocks collect enough data for the set-similarity model to work. Some of this was added by me by hand.

If a word is similar to multiple words in a given set, it could be considered to be apart of the set. This similarity is simply given by <a href="https://en.wikipedia.org/wiki/Levenshtein_distance">Levenshtein distance</a>, accomplished by the `edit_distance` function of the `nltk` package.

This Notebook also contains the final Neural Network for numbers, as I have decided earlier not to use a Neural Network model for them, but later changed my mind.

All results from this Notebook are also saved separately.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import random
import math
random_state = 777
import nltk

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


#### My plans can be summed up the following way:

- ADV -> Határozószó, set-similarity
- ART -> Névelő, LIST: a, az, egy. (Definite / Indefinite)
- CONJ -> Kötőszó, set-similarity
- DET -> Névmás, set-similarity + DET NN

- NUM -> Számlálószó, NUM NN
- NUM[AGGREG] -> Számlálószó, (gyűjt.), NUM NN
- NUM[MULTIPL] -> Számlálószó (többsz.), NUM NN
- NUM[ORD-ITER-ACCOMPL] -> Számlálószó (rend.) NUM NN

- ONO -> Hangutánzó, set-similarity
- POSTP -> Névutó, LIST + set-similarity
- PREP -> Elöljáró, LIST
- PREV -> Igekötő, LIST + set-similarity
- UTT-INT -> Töltelékszó, set-similarity

In [3]:
adverbs = pd.read_csv('adverbs.csv')

In [4]:
adverbs

,word,stem,morphemes
0,abszolút,abszolút,ADV
1,abszolúte,abszolúte,ADV
2,addig-ameddig,addig-ameddig,ADV
3,addiglan,addiglan,ADV
4,addigra,addigra,ADV
...,...,...,...
2121,ültében,ültében,ADV
2122,ültéből,ültéből,ADV
2123,ünnepnap,ünnepnap,ADV
2124,üptre,üptre,ADV


In [5]:
adverbs_list = list(adverbs['word'])

In [6]:
def_arts_list = ['a', 'az']
indef_arts_list = ['egy']

In [7]:
conjs = pd.read_csv('conjs.csv')

In [8]:
conjs_list = list(conjs['word'])

In [9]:
dets = pd.read_csv('dets.csv')

In [10]:
dets_list = list(dets['word'])

In [11]:
dets_list

['ama',
 'azon',
 'egyazon',
 'egyugyanazon',
 'eme',
 'emez',
 'ezen',
 'mindama',
 'mindaz',
 'mindazon',
 'mindeme',
 'mindezen',
 'ugyanazon',
 'ugyanezen']

In [12]:
new_dets = 'egyugyanaz egyugyanez egyugyanezen egyugyanakként egyugyanekként ugyanaz ugyanez ugyanakként ugyanekként mindez e ugyanakkor ugyanekkor egyugyanakkor egyugyanekkor'.split(' ')

In [13]:
dets_list += new_dets

In [14]:
dets_list.sort()

In [15]:
dets_list

['ama',
 'azon',
 'e',
 'egyazon',
 'egyugyanakkor',
 'egyugyanakként',
 'egyugyanaz',
 'egyugyanazon',
 'egyugyanekkor',
 'egyugyanekként',
 'egyugyanez',
 'egyugyanezen',
 'eme',
 'emez',
 'ezen',
 'mindama',
 'mindaz',
 'mindazon',
 'mindeme',
 'mindez',
 'mindezen',
 'ugyanakkor',
 'ugyanakként',
 'ugyanaz',
 'ugyanazon',
 'ugyanekkor',
 'ugyanekként',
 'ugyanez',
 'ugyanezen']

In [16]:
nums_aggreg = pd.read_csv('nums_aggreg.csv')

In [17]:
nums_aggreg_list = list(nums_aggreg['word'])

In [18]:
nums_aggreg_list

['ahányan',
 'akárhanyan',
 'akárhányan',
 'annyian',
 'bárhányan',
 'csomóan',
 'egyen',
 'egynéhányan',
 'egypáran',
 'ennyien',
 'ezren',
 'feleannyian',
 'feleennyien',
 'félannyian',
 'hanyan',
 'harmincan',
 'hatan',
 'hatvanan',
 'heten',
 'hetvenen',
 'hányan',
 'húszan',
 'jónéhányan',
 'kevesen',
 'kilencen',
 'kismillióan',
 'kétannyian',
 'megannyian',
 'milliárdan',
 'millióan',
 'mindahányan',
 'másfelen',
 'negyvenen',
 'nyolcan',
 'nyolcvanan',
 'négyen',
 'néhányan',
 'rengetegen',
 'sehányan',
 'sokan',
 'számosan',
 'százan',
 'tengernyien',
 'tízen',
 'ugyanannyian',
 'ugyanennyien',
 'valahányan',
 'öten',
 'öten',
 'öten',
 'édeskevesen',
 'öten',
 'ötvenen']

In [19]:
new_nums_aggreg = 'akármennyien ketten hárman négyen öten hatan heten nyolcan kilencen tizen tizenegyen huszonhatan harminckilencen negyvenöten ötvenheten hatvanketten hetvennyolcan nyolcvannégyen kilencvenhárman sokszázan sokezren sokszázezren sokmillióan sokszázmillióan százmilióan sokmillárdan sokbillióan sokbilliárdan billiárdan billióan soktrillióan soktrilliárdan trilliárdan trillióan páran jópáran'.split(' ')

In [20]:
nums_aggreg_list += new_nums_aggreg
nums_aggreg_list = list(set(nums_aggreg_list))
nums_aggreg_list.sort()

In [21]:
nums_aggreg_list

['ahányan',
 'akárhanyan',
 'akárhányan',
 'akármennyien',
 'annyian',
 'billiárdan',
 'billióan',
 'bárhányan',
 'csomóan',
 'egyen',
 'egynéhányan',
 'egypáran',
 'ennyien',
 'ezren',
 'feleannyian',
 'feleennyien',
 'félannyian',
 'hanyan',
 'harmincan',
 'harminckilencen',
 'hatan',
 'hatvanan',
 'hatvanketten',
 'heten',
 'hetvenen',
 'hetvennyolcan',
 'huszonhatan',
 'hányan',
 'hárman',
 'húszan',
 'jónéhányan',
 'jópáran',
 'ketten',
 'kevesen',
 'kilencen',
 'kilencvenhárman',
 'kismillióan',
 'kétannyian',
 'megannyian',
 'milliárdan',
 'millióan',
 'mindahányan',
 'másfelen',
 'negyvenen',
 'negyvenöten',
 'nyolcan',
 'nyolcvanan',
 'nyolcvannégyen',
 'négyen',
 'néhányan',
 'páran',
 'rengetegen',
 'sehányan',
 'sokan',
 'sokbilliárdan',
 'sokbillióan',
 'sokezren',
 'sokmillióan',
 'sokmillárdan',
 'sokszázan',
 'sokszázezren',
 'sokszázmillióan',
 'soktrilliárdan',
 'soktrillióan',
 'számosan',
 'százan',
 'százmilióan',
 'tengernyien',
 'tizen',
 'tizenegyen',
 'trilliár

In [22]:
nums_multipl = pd.read_csv('nums_multipl.csv')

In [23]:
nums_multipl_list = list(nums_multipl['word'])

In [24]:
nums_multipl_list

['ahányszorta',
 'annyiszorta',
 'egyszerte',
 'hatszorta',
 'hányszorta',
 'háromszorta',
 'hétszerte',
 'hússzorta',
 'kétszerte',
 'milliárdszorta',
 'másfélszerte',
 'nyolcszorta',
 'négyszerte',
 'tízszerte',
 'ötszörte']

In [25]:
new_nums_multipl = 'kétszerte négyszerte hétszerte kilencszerte százszorta ezerszerte miliószorta millárdszorta billiószorta billiárdszorta trilliószorta trilliárdszorta ötvenhetedszerte nyolcvankilencedszerte tizennegyedszerte hatvanegyedszerte harmincötödszörte hetvenkettedszerte negyvenhatdoszorta huszonharmadszorta kilencvennyolcadszorta'.split(' ')

In [26]:
nums_multipl_list += new_nums_multipl
nums_multipl_list.sort()

In [27]:
nums_multipl_list

['ahányszorta',
 'annyiszorta',
 'billiárdszorta',
 'billiószorta',
 'egyszerte',
 'ezerszerte',
 'harmincötödszörte',
 'hatszorta',
 'hatvanegyedszerte',
 'hetvenkettedszerte',
 'huszonharmadszorta',
 'hányszorta',
 'háromszorta',
 'hétszerte',
 'hétszerte',
 'hússzorta',
 'kilencszerte',
 'kilencvennyolcadszorta',
 'kétszerte',
 'kétszerte',
 'miliószorta',
 'milliárdszorta',
 'millárdszorta',
 'másfélszerte',
 'negyvenhatdoszorta',
 'nyolcszorta',
 'nyolcvankilencedszerte',
 'négyszerte',
 'négyszerte',
 'százszorta',
 'tizennegyedszerte',
 'trilliárdszorta',
 'trilliószorta',
 'tízszerte',
 'ötszörte',
 'ötvenhetedszerte']

In [28]:
nums_iter = pd.read_csv('nums_iter.csv')

In [29]:
nums_iter_list = list(nums_iter['word'])

In [30]:
nums_iter_list

['akárhányadszorra',
 'hanyadszorra',
 'harmadszorra',
 'harmincadszorra',
 'hatodszorra',
 'hatvanadszorra',
 'huszadszorra',
 'hányadszorra',
 'milliomodszorra',
 'másodszorra',
 'nyolcadszorra',
 'sokadszorra',
 'századszorra']

In [31]:
new_nums_iter = 'előszörre elsőre negyedszerre ötödszörre hetedszerre nyolcadszorra kilencedszerre tizedszerre századszorra ezredszerre milliárdadszorra billomodszorra billiárdadszorra trilliomodszorra trilliárdadszorra negyvenhatodszorra kilencvenharmadszorra huszonegyedszerre tizennegyedszerre nyolcvankettedszerre hetvenhetedszerre hatvankilencedszerre ötvennyolcadszorra harmincötödszörre'.split(' ')

In [35]:
nums_iter_list += new_nums_iter
nums_iter_list.sort()

In [36]:
nums_iter_list

['akárhányadszorra',
 'billiárdadszorra',
 'billomodszorra',
 'elsőre',
 'előszörre',
 'ezredszerre',
 'hanyadszorra',
 'harmadszorra',
 'harmincadszorra',
 'harmincötödszörre',
 'hatodszorra',
 'hatvanadszorra',
 'hatvankilencedszerre',
 'hetedszerre',
 'hetvenhetedszerre',
 'huszadszorra',
 'huszonegyedszerre',
 'hányadszorra',
 'kilencedszerre',
 'kilencvenharmadszorra',
 'milliomodszorra',
 'milliárdadszorra',
 'másodszorra',
 'negyedszerre',
 'negyvenhatodszorra',
 'nyolcadszorra',
 'nyolcadszorra',
 'nyolcvankettedszerre',
 'sokadszorra',
 'századszorra',
 'századszorra',
 'tizedszerre',
 'tizennegyedszerre',
 'trilliomodszorra',
 'trilliárdadszorra',
 'ötvennyolcadszorra',
 'ötödszörre']

In [37]:
nums = pd.read_csv('nums.csv')
nums.drop('Unnamed: 0', axis=1, inplace=True)

In [38]:
nums_list = list(set(nums['stem']))
nums_list.sort()
nums_list

['ahány',
 'akárhány',
 'annyi',
 'bárhány',
 'csomó',
 'egy',
 'egynéhány',
 'egypár',
 'ennyi',
 'ezer',
 'feleannyi',
 'feleennyi',
 'félannyi',
 'harminc',
 'hat',
 'hatvan',
 'hetven',
 'hány',
 'három',
 'hét',
 'húsz',
 'jónéhány',
 'kettő',
 'kevés',
 'kilenc',
 'kismillió',
 'kétannyi',
 'megannyi',
 'milliárd',
 'millió',
 'mindahány',
 'másfél',
 'negyven',
 'nyolc',
 'nyolcvan',
 'négy',
 'néhány',
 'rengeteg',
 'sehány',
 'sok',
 'számos',
 'száz',
 'tengernyi',
 'tíz',
 'ugyanannyi',
 'ugyanennyi',
 'valahány',
 'édeskevés',
 'öt',
 'ötven']

In [39]:
new_nums = 'kilencvenöt nyolcvannégy ötvenhét harvanhárom huszonhat harmicnyolc tizenegy hetvenkilenc negyvenkettő sokszáz sokezer sokszázezer sokmilió sokmilliárd billió sokbillió billiárd sokbilliárd trillió soktrillió trilliárd soktrilliárd többszáz többezer többmillió többmilliárd többtrillió többtrilliárd'.split(' ')

In [40]:
nums_list += new_nums
nums_list.sort()

In [41]:
nums_list

['ahány',
 'akárhány',
 'annyi',
 'billiárd',
 'billió',
 'bárhány',
 'csomó',
 'egy',
 'egynéhány',
 'egypár',
 'ennyi',
 'ezer',
 'feleannyi',
 'feleennyi',
 'félannyi',
 'harmicnyolc',
 'harminc',
 'harvanhárom',
 'hat',
 'hatvan',
 'hetven',
 'hetvenkilenc',
 'huszonhat',
 'hány',
 'három',
 'hét',
 'húsz',
 'jónéhány',
 'kettő',
 'kevés',
 'kilenc',
 'kilencvenöt',
 'kismillió',
 'kétannyi',
 'megannyi',
 'milliárd',
 'millió',
 'mindahány',
 'másfél',
 'negyven',
 'negyvenkettő',
 'nyolc',
 'nyolcvan',
 'nyolcvannégy',
 'négy',
 'néhány',
 'rengeteg',
 'sehány',
 'sok',
 'sokbilliárd',
 'sokbillió',
 'sokezer',
 'sokmilió',
 'sokmilliárd',
 'sokszáz',
 'sokszázezer',
 'soktrilliárd',
 'soktrillió',
 'számos',
 'száz',
 'tengernyi',
 'tizenegy',
 'trilliárd',
 'trillió',
 'tíz',
 'többezer',
 'többmilliárd',
 'többmillió',
 'többszáz',
 'többtrilliárd',
 'többtrillió',
 'ugyanannyi',
 'ugyanennyi',
 'valahány',
 'édeskevés',
 'öt',
 'ötven',
 'ötvenhét']

In [42]:
L = list('123456789')
M = list('123456789')
random.shuffle(L)
random.shuffle(M)
print(' '.join([L[i] + M[i] for i in range(len(L))]))

19 65 74 47 26 82 38 51 93


In [43]:
nums_order_list = 'első második harmadik negyedik ötödik hatodik hetedik nyolcadik kilencedik tizedik hányadik akárhányadik valahányadik sokadik többedik negyvenkettedik ötvenegyedik tizenkilencedik hetvenötödik huszonnegyedik hatvanharmadik harminchatodik kilencvennyolcadik nyolcvanhetedik századik sokszázadik többszázadik ezredik sokezredik többezredik milliomodik sokmilliomodik többmilliomodik millárdadik sokmillárdadik többmillárdadik billiomodik sokbilliomodik billárdadik sokbillárdadik trilliomodik soktrilliomodik többtrilliomodik trillárdadik soktrillárdadik többtrillárdadik'.split(' ')

In [44]:
nums_order_list

['első',
 'második',
 'harmadik',
 'negyedik',
 'ötödik',
 'hatodik',
 'hetedik',
 'nyolcadik',
 'kilencedik',
 'tizedik',
 'hányadik',
 'akárhányadik',
 'valahányadik',
 'sokadik',
 'többedik',
 'negyvenkettedik',
 'ötvenegyedik',
 'tizenkilencedik',
 'hetvenötödik',
 'huszonnegyedik',
 'hatvanharmadik',
 'harminchatodik',
 'kilencvennyolcadik',
 'nyolcvanhetedik',
 'századik',
 'sokszázadik',
 'többszázadik',
 'ezredik',
 'sokezredik',
 'többezredik',
 'milliomodik',
 'sokmilliomodik',
 'többmilliomodik',
 'millárdadik',
 'sokmillárdadik',
 'többmillárdadik',
 'billiomodik',
 'sokbilliomodik',
 'billárdadik',
 'sokbillárdadik',
 'trilliomodik',
 'soktrilliomodik',
 'többtrilliomodik',
 'trillárdadik',
 'soktrillárdadik',
 'többtrillárdadik']

In [45]:
nums_extended = pd.DataFrame([], columns=['word', 'regular', 'aggreg', 'multipl', 'order', 'iter'])

In [121]:
for n in nums_list:
    nums_extended = nums_extended.append(
        {'word': n, 'regular': 1,  'aggreg': 0, 'multipl': 0, 'order': 0, 'iter': 0},
        ignore_index=True)
for n in nums_aggreg_list:
    nums_extended = nums_extended.append(
        {'word': n, 'regular': 0,  'aggreg': 1, 'multipl': 0, 'order': 0, 'iter': 0},
        ignore_index=True)
for n in nums_multipl_list:
    nums_extended = nums_extended.append(
        {'word': n, 'regular': 0,  'aggreg': 0, 'multipl': 1, 'order': 0, 'iter': 0},
        ignore_index=True)
for n in nums_order_list:
    nums_extended = nums_extended.append(
        {'word': n, 'regular': 0,  'aggreg': 0, 'multipl': 0, 'order': 1, 'iter': 0},
        ignore_index=True)
for n in nums_iter_list:
    nums_extended = nums_extended.append(
        {'word': n, 'regular': 0,  'aggreg': 0, 'multipl': 0, 'order': 0, 'iter': 1},
        ignore_index=True)

In [122]:
nums_extended

,word,regular,aggreg,multipl,order,iter
0,ahány,1,0,0,0,0
1,akárhány,1,0,0,0,0
2,annyi,1,0,0,0,0
3,billiárd,1,0,0,0,0
4,billió,1,0,0,0,0
...,...,...,...,...,...,...
272,tizennegyedszerre,0,0,0,0,1
273,trilliomodszorra,0,0,0,0,1
274,trilliárdadszorra,0,0,0,0,1
275,ötvennyolcadszorra,0,0,0,0,1


In [123]:
nums_extended_index = list(range(len(nums_extended)))
random.shuffle(nums_extended_index)

In [124]:
nums_extended_index

[251,
 79,
 25,
 83,
 225,
 205,
 58,
 201,
 217,
 198,
 272,
 65,
 10,
 5,
 160,
 196,
 69,
 41,
 207,
 159,
 48,
 213,
 188,
 112,
 95,
 239,
 139,
 3,
 101,
 27,
 200,
 8,
 12,
 31,
 74,
 34,
 38,
 226,
 264,
 1,
 228,
 210,
 230,
 221,
 240,
 170,
 214,
 174,
 189,
 255,
 237,
 182,
 15,
 17,
 270,
 166,
 82,
 195,
 71,
 45,
 96,
 262,
 52,
 108,
 167,
 66,
 149,
 109,
 152,
 256,
 13,
 242,
 24,
 76,
 173,
 97,
 254,
 125,
 238,
 70,
 219,
 110,
 46,
 163,
 177,
 59,
 216,
 235,
 117,
 84,
 192,
 140,
 151,
 23,
 51,
 6,
 80,
 194,
 61,
 62,
 87,
 116,
 131,
 92,
 105,
 218,
 40,
 268,
 9,
 187,
 49,
 231,
 273,
 128,
 249,
 75,
 43,
 261,
 137,
 153,
 0,
 275,
 37,
 100,
 47,
 124,
 33,
 60,
 222,
 165,
 54,
 77,
 178,
 103,
 28,
 44,
 11,
 22,
 120,
 136,
 203,
 64,
 158,
 135,
 267,
 72,
 16,
 245,
 115,
 89,
 229,
 132,
 236,
 32,
 107,
 215,
 111,
 150,
 234,
 14,
 114,
 102,
 99,
 94,
 265,
 257,
 202,
 186,
 42,
 29,
 7,
 176,
 26,
 134,
 106,
 119,
 93,
 53,
 141,
 18,
 15

In [127]:
nums_extended = nums_extended.reindex(nums_extended_index)
nums_extended.reset_index(inplace = True, drop = True)

In [128]:
nums_extended

,word,regular,aggreg,multipl,order,iter
0,hatvanadszorra,0,0,0,0,1
1,akárhanyan,0,1,0,0,0
2,hét,1,0,0,0,0
3,billiárdan,0,1,0,0,0
4,sokmilliomodik,0,0,0,1,0
...,...,...,...,...,...,...
272,billiárdadszorra,0,0,0,0,1
273,annyi,1,0,0,0,0
274,édeskevesen,0,1,0,0,0
275,harmincadszorra,0,0,0,0,1


In [129]:
nums_extended.to_csv('nums_extended.csv')

In [46]:
nums_extended = pd.read_csv('nums_extended.csv')

In [47]:
chars = " 0123456789.:,;!%&'*_-=~\\()|[]{}aáäbcdeéfghiíjklmnoóöőpqrstuúüűvwxyz"
encode_dict = {}
decode_dict = {}

for c in range(len(chars)):
    encode_dict[chars[c]] = c
    decode_dict[c] = chars[c]

def encode(w):
    ret = []
    for c in w:
        ret.append(encode_dict[c])
    return np.array(ret)

def decode(a):
    ret = []
    for i in a:
        ret.append(decode_dict[i])
    return ''.join(ret)

M = len(encode_dict)
W = 44
def one_hot_encode(w):
    e = encode(w)
    ohe = np.zeros((W, M))
    ohe[np.arange(len(e)),e] = 1
    return ohe 

def one_hot_encode_col(col):
    return np.array([one_hot_encode(w) for w in col])

In [131]:
nums_X = one_hot_encode_col(list(nums_extended['word']))
nums_y = np.array(nums_extended[nums_extended.columns.difference(['word'])])

In [134]:
nums_X.shape, nums_y.shape

((277, 44, 68), (277, 5))

In [207]:
inp = tf.keras.Input(shape=(44,68), name='input')

def conv_pool_layer(kernel_size,pool_size,input_layer,name):
    
    conv = tf.keras.layers.Conv1D(68,kernel_size=kernel_size,
        input_shape=(44,68), activation='relu', padding='same',
        kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
        bias_regularizer=regularizers.l2(1e-3),
        activity_regularizer=regularizers.l2(1e-4),
        data_format='channels_last', name='1D_conv' + name)(input_layer)
    
    pool = max_pool3 = tf.keras.layers.MaxPooling1D(pool_size=pool_size,
        data_format='channels_first', name= '1D_maxpool' + name)(conv)

    return pool

model1 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(1,1,inp,'1'))
model3 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(3,2,inp,'3'))
model5 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(5,2,inp,'5'))
model7 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(7,4,inp,'7'))
model9 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(9,4,inp,'9'))

combined = tf.keras.layers.concatenate([model1.output, model3.output, model5.output,
                                        model7.output, model9.output], name='concatenate')

flat = tf.keras.layers.Flatten(name='flatten')(combined)
drop = tf.keras.layers.Dropout(0.57,name='droput_0.57')(flat)
dense5 = tf.keras.layers.Dense(5, activation='softmax',name='dense3')(drop)

nums_conv_model2 = tf.keras.Model(inputs=[model1.input],
                            outputs=dense5,name='Nums_Multiconv_Model2')
nums_conv_model2.output_shape

(None, 5)

In [208]:
nums_conv_model2.compile(optimizer='adam', loss='categorical_crossentropy')

In [209]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

In [212]:
nums_conv_model2.fit(nums_X, nums_y, epochs=300, batch_size=20, callbacks=[early_stopping], validation_split=0.2)

Train on 221 samples, validate on 56 samples
Epoch 1/300
221/221 [==============================] - 3s 16ms/sample - loss: 2.3435 - val_loss: 2.1811
Epoch 2/300
221/221 [==============================] - 1s 5ms/sample - loss: 2.0029 - val_loss: 1.9589
Epoch 3/300
221/221 [==============================] - 1s 5ms/sample - loss: 1.7590 - val_loss: 1.7967
Epoch 4/300
221/221 [==============================] - 1s 5ms/sample - loss: 1.5479 - val_loss: 1.6782
Epoch 5/300
221/221 [==============================] - 1s 5ms/sample - loss: 1.3792 - val_loss: 1.5775
Epoch 6/300
221/221 [==============================] - 1s 5ms/sample - loss: 1.2619 - val_loss: 1.4638
Epoch 7/300
221/221 [==============================] - 1s 5ms/sample - loss: 1.1404 - val_loss: 1.3373
Epoch 8/300
221/221 [==============================] - 1s 5ms/sample - loss: 1.0372 - val_loss: 1.2646
Epoch 9/300
221/221 [==============================] - 1s 5ms/sample - loss: 0.9528 - val_loss: 1.2327
Epoch 10/300
221/221 [=====

In [213]:
nums_conv_model2.save('nums_conv_model_extended2.h5')

In [214]:
num_predict_dict = {0: 'aggreg', 1: 'iter', 2: 'multipl', 3: 'order', 4: 'regular'}

In [223]:
def predict_num_type(n):
    nn_input = one_hot_encode_col([n[max(0,len(n)-10):]])
    pred = nums_conv_model2.predict(nn_input)[0]
    print(pred)
    print(num_predict_dict[np.argmax(pred)])

In [224]:
predict_num_type('nyolcvanegy')

[5.1048491e-03 8.8301644e-04 1.3392670e-03 3.1497085e-03 9.8952311e-01]
regular


In [230]:
predict_num_type('hatszázhetvenhetedszerre')

[0.01077529 0.90196913 0.07592694 0.00481468 0.00651402]
iter


In [231]:
predict_num_type('milliomodik')

[5.3703872e-05 5.2945520e-04 4.2699877e-04 9.8588330e-01 1.3106586e-02]
order


In [48]:
onos = pd.read_csv('onos.csv')

In [49]:
onos_list = list(onos['word'])
onos_list

['bee',
 'brekeke',
 'brr',
 'bruhaha',
 'brumm-brumm',
 'bumm',
 'bú',
 'c',
 'cic',
 'cicic',
 'csatt',
 'csinn-bumm',
 'csirip',
 'csitt-csatt',
 'dirr',
 'dirr-durr',
 'durr',
 'giling-galang',
 'hah',
 'haha',
 'hahaha',
 'heh',
 'hehe',
 'hehehe',
 'hihi',
 'hihihi',
 'háp-háp',
 'höhö',
 'höhöhö',
 'iá',
 'kakukk',
 'kipp',
 'kipp-kopp',
 'kitty-kotty',
 'kopp',
 'kopp-kopp',
 'kotkodács',
 'kotty',
 'kuc-kuc',
 'kukk',
 'kukurikú',
 'kurutty',
 'kutykurutty',
 'kár-kár',
 'lalala',
 'mek-mek',
 'miau',
 'nyekk',
 'nyihaha',
 'oá',
 'paff',
 'picsi-pacsi',
 'piff',
 'piff-paff',
 'piff-puff',
 'pipi',
 'potty',
 'puff',
 'püff',
 'reccs',
 'röf-röf',
 'sipp-supp',
 'sutty',
 'tam-tam',
 'tik-tak',
 'trotty',
 'tyutyu',
 'tánci-tánci',
 'töf-töf',
 'tütü',
 'u',
 'vau-vau',
 'zsipp-zsupp',
 'zsupp',
 'zsupsz',
 'zupp',
 'zutty',
 'züm-züm',
 'ú',
 'ú']

In [50]:
postps = pd.read_csv('postps.csv')

In [51]:
postps_list = list(set(postps['word']))
postps_list.sort()
postps_list

['adódóan',
 'alapján',
 'alatt',
 'alul',
 'alá',
 'alól',
 'bekövetkeztével',
 'belül',
 'bezárólag',
 'beálltával',
 'dacára',
 'egyetemben',
 'egyidőben',
 'együtt',
 'ellen',
 'ellentétben',
 'ellenébe',
 'ellenében',
 'ellenére',
 'elmúltával',
 'elteltével',
 'eltérően',
 'elé',
 'elől',
 'előrehaladtával',
 'előtt',
 'eredményeképpen',
 'eredően',
 'erejéig',
 'esetén',
 'ezelőtt',
 'fakadóan',
 'fakadólag',
 'fejében',
 'fele',
 'felett',
 'felé',
 'felül',
 'felől',
 'fogva',
 'fogvást',
 'folyamán',
 'folytán',
 'folyólag',
 'fölé',
 'fölött',
 'fölül',
 'függetlenül',
 'függően',
 'gyanánt',
 'hallatán',
 'hallatára',
 'helyett',
 'hiányában',
 'hosszat',
 'hála',
 'híján',
 'illetően',
 'indíttatva',
 'innen',
 'iránt',
 'jóvoltából',
 'kapcsán',
 'keresztül',
 'kezdve',
 'kifolyólag',
 'képest',
 'kívül',
 'köré',
 'körül',
 'köszönhetően',
 'következtében',
 'következőleg',
 'követően',
 'közben',
 'közeledtével',
 'közepett',
 'közepette',
 'közt',
 'közé',
 'között',
 

In [52]:
preps = pd.read_csv('preps.csv')

In [53]:
preps_list = list(preps['word'])
preps_list

['alias', 'kivéve', 'kontra', 'pró']

In [54]:
preps_list += ['pro']

In [55]:
prevs = pd.read_csv('prevs.csv')

In [56]:
prevs_list = list(set(prevs['word']))
prevs_list.sort()
prevs_list

['abba',
 'agyon',
 'alul',
 'alá',
 'be',
 'bele',
 'bé',
 'békén',
 'cserben',
 'egybe',
 'egyet',
 'együtt',
 'el',
 'ellen',
 'ellent',
 'elé',
 'elő',
 'előre',
 'fel',
 'felül',
 'fenn',
 'félbe',
 'félre',
 'föl',
 'földet',
 'följebb',
 'fölé',
 'haza',
 'helyben',
 'helyre',
 'helyt',
 'hoppon',
 'hozzá',
 'hátra',
 'ide',
 'jól',
 'jót',
 'jóvá',
 'karban',
 'keresztül',
 'ketté',
 'ki',
 'kijjebb',
 'kinn',
 'kétségbe',
 'kölcsön',
 'körbe',
 'körül',
 'közbe',
 'közre',
 'közzé',
 'külön',
 'le',
 'létre',
 'meg',
 'mellé',
 'nagyot',
 'neki',
 'nyilván',
 'oda',
 'ott',
 'pofon',
 'pórul',
 'rajta',
 'rendre',
 'rá',
 'szabadjára',
 'szabadlábra',
 'szembe',
 'szemre',
 'szemügyre',
 'szerte',
 'számba',
 'széjjel',
 'széjjelebb',
 'szét',
 'szörnyet',
 'síkra',
 'tele',
 'teli',
 'torkig',
 'tovább',
 'tudomásul',
 'tudtul',
 'tönkre',
 'túl',
 'utol',
 'utána',
 'vissza',
 'végbe',
 'véghez',
 'végig',
 'végre',
 'zokon',
 'zöldágra',
 'által',
 'át',
 'észhez',
 'észre'

In [57]:
utt_ints = pd.read_csv('utt_ints.csv')

In [58]:
utt_ints_list = list(set(utt_ints['word']))
utt_ints_list.sort()
utt_ints_list

['abcúg',
 'addsza',
 'adieu',
 'adjonisten',
 'adta',
 'agyő',
 'ah',
 'aha',
 'aham',
 'ahá',
 'ahán',
 'aj',
 'ajaj',
 'ajjaj',
 'ajvé',
 'alleluja',
 'alászolgája',
 'aló',
 'annyi',
 'au',
 'azám',
 'aú',
 'bah',
 'bakfitty',
 'bakker',
 'bamm',
 'bamme',
 'beh',
 'bim-bam',
 'bimm',
 'bizonyisten',
 'bravó',
 'bá',
 'ciao',
 'co',
 'coki',
 'csaó',
 'csend',
 'csiba',
 'csicsíja',
 'csigavér',
 'csingilingi',
 'csiribí-csiribá',
 'csiribú-csiribá',
 'csitt',
 'csocsi',
 'csuhaj',
 'csá',
 'csákó',
 'császtok',
 'csáó',
 'csók',
 'csönd',
 'csüccs',
 'cső',
 'csősztök',
 'curukk',
 'dehát',
 'dádá',
 'dícsértessék',
 'e',
 'egy-kettő',
 'egykutya',
 'egészség',
 'egészségedre',
 'egészségetekre',
 'egészségére',
 'egészségükre',
 'egészségünkre',
 'eh',
 'ej',
 'ej-haj',
 'ejde',
 'ejej',
 'ejha',
 'ejnye',
 'ejnye-bejnye',
 'ejnyebejnye',
 'előre',
 'ennyi',
 'f',
 'fi',
 'figyelem',
 'finis',
 'fuccs',
 'fuj',
 'fujj',
 'fékomadta',
 'fú',
 'fúj',
 'gyerünk',
 'gyí',
 'ha',
 'ha

In [59]:
all_nums_list = list(nums_extended['word'])
all_nums_list.sort()
all_nums_list

['ahány',
 'ahányan',
 'ahányszorta',
 'akárhanyan',
 'akárhány',
 'akárhányadik',
 'akárhányadszorra',
 'akárhányan',
 'akármennyien',
 'annyi',
 'annyian',
 'annyiszorta',
 'billiomodik',
 'billiárd',
 'billiárdadszorra',
 'billiárdan',
 'billiárdszorta',
 'billió',
 'billióan',
 'billiószorta',
 'billomodszorra',
 'billárdadik',
 'bárhány',
 'bárhányan',
 'csomó',
 'csomóan',
 'egy',
 'egyen',
 'egynéhány',
 'egynéhányan',
 'egypár',
 'egypáran',
 'egyszerte',
 'első',
 'elsőre',
 'előszörre',
 'ennyi',
 'ennyien',
 'ezer',
 'ezerszerte',
 'ezredik',
 'ezredszerre',
 'ezren',
 'feleannyi',
 'feleannyian',
 'feleennyi',
 'feleennyien',
 'félannyi',
 'félannyian',
 'hanyadszorra',
 'hanyan',
 'harmadik',
 'harmadszorra',
 'harmicnyolc',
 'harminc',
 'harmincadszorra',
 'harmincan',
 'harminchatodik',
 'harminckilencen',
 'harmincötödszörre',
 'harmincötödszörte',
 'harvanhárom',
 'hat',
 'hatan',
 'hatodik',
 'hatodszorra',
 'hatszorta',
 'hatvan',
 'hatvanadszorra',
 'hatvanan',
 'ha

In [60]:
def word_similarity(w1, w2):
    return nltk.edit_distance(w1, w2) / max(len(w1),len(w2))

In [61]:
def set_similarity(word, word_set, word_set_avg_len=None):
    sim_full_sum = 0
    sim_end_sum = 0
    counter = 0
    
    if word_set_avg_len is not None:
        avg_len = word_set_avg_len
    else:
        avg_len = int(np.mean([len(w) for w in word_set]))

    for w in word_set:
        sim_full = 1 - word_similarity(word, w)
        sim_end = 1 -  word_similarity(word[max(0,len(word)-avg_len):], w)
        if sim_full > 0.5:
            counter += 1
            sim_full_sum += sim_full
        if sim_end > 0.5:
            counter += 1
            sim_end_sum += sim_end
    
    return ((sim_full_sum + sim_end_sum) / 2) * (counter / len(word_set))

In [62]:
set_similarity('addigra', adverbs_list)

0.01088563365139094

In [343]:
set_similarity('eddigre', adverbs_list)

0.021267302781884153

In [242]:
set_similarity('megszentségteleníthetetlenségeskedéseitekért', adverbs_list)

0.001883334079947438

In [243]:
set_similarity('tlenül', adverbs_list)

0.12443281083920779

In [244]:
set_similarity('megszentségteleníthetetlenül', adverbs_list)

0.16371603662878362

In [251]:
set_similarity('látva', adverbs_list)

felváltva
felváltva
hátra
hátra
játszva
játszva
lopva
lopva


0.009820961937612924